In [1]:
import itertools
    
def bruhat_decomp_tokens(rank, flip_index):
    index = rank - flip_index + 1
    cell_dimensions = []

    range_of_first_counter = rank-index + 1
    token_counter_ranges = [range(2*index + k) for k in range(rank-index+1)]
    token_counter_possibilities = itertools.product(*token_counter_ranges)

    token_counter_valid = []

    for possibility in token_counter_possibilities:
        flag = True

        for i in range(len(possibility)-1):
            if possibility[i+1] > possibility[i] + 1:
                flag = False
                break
            if possibility[i] <= index + i - 1 and possibility[i+1] > possibility[i]:
                flag = False
                break

        if flag == True:
            token_counter_valid.append(possibility)

    return token_counter_valid  
    
def dimensions_of_cells(shortest_elements):
    y = [x.length() for x in shortest_elements]
    y.sort()
    return y


# This only works for bundles where only one weight has a negative which is dominant! Luckily that is all we need
def find_dual_rep(bundle):
    weights = bundle.weight_multiplicities()
    highest_weight_of_dual = 0
    for weight in weights:
        if (-weight).is_dominant():
            highest_weight_of_dual = -weight
            break
    return WeylCharacterRing(bundle.cartan_type())(highest_weight_of_dual)

def weyl_action(weight, i, cartan_matrix):
    return [x-y for x,y in zip(weight,cartan_matrix.columns()[i] * weight[i])]

def to_fw_basis(weight):
    reversed_fw = []
    temp_weight = list(weight)
    for i in range(len(temp_weight)):
        entry = temp_weight.pop()
        reversed_fw.append(entry)
        temp_weight = [x-entry for x in temp_weight]
        
    return reversed_fw[::-1]

def to_L_basis(weight):
    fw_weight = [0 for j in range(len(weight))]
    for i in range(len(weight)):
        fw_weight_add  = [weight[i] for j in range(i+1)]
        for j in range(len(weight) - i-1):
            fw_weight_add.append(0)
        fw_weight = [a + b for a,b in zip(fw_weight_add, fw_weight)]
    return fw_weight

def borel_weil_bott(group_type, weight):
    rk = len(weight)
    cartan_matrix = CartanMatrix([group_type, rk])
    
    weight = to_fw_basis(weight)
    degree = 0
    weight = [x+1 for x in weight]
    
    while any(j < 0 for j in weight):
        for i in range(len(weight)):
            if weight[i] < 0:
                weight = weyl_action(weight, i, cartan_matrix);
                degree += 1

    weight = [x-1 for x in weight]

    if not any(j < 0 for j in weight):
        weight_in_L_basis = to_L_basis(weight)
        return [WeylCharacterRing([group_type, rk])(*weight_in_L_basis).degree(), degree]
    else:
        return "No cohomology"

1. Choose case to work on and then calculate the rank and corresponding maximal parabolic to consider



In [2]:
r = 2

rank = 3*r - 1
maximal_parabolic_indices = [2*r-1,2*r]
cartan_type = CartanType(["C", rank])

2. Set root system, Weyl group, character ring, etc.

In [3]:
root_system = RootSystem(cartan_type)
root_space = root_system.ambient_space()
weyl_group = root_space.weyl_group()
char_ring = WeylCharacterRing(cartan_type)
weight_ring = WeightRing(char_ring)

fundamental_weights = root_space.fundamental_weights()

3. Find Bruhat Cells

In [4]:
maximal_parabolic_bruhat_cells = [bruhat_decomp_tokens(rank, index) for index in maximal_parabolic_indices]

4. Compare Bruhat Decompositions

In [5]:
sorted([sum(token) for token in maximal_parabolic_bruhat_cells[0]]) == sorted([sum(token) for token in maximal_parabolic_bruhat_cells[1]])

True

5. Set character rings of parabolics and find the pushforward of the line bundle

In [6]:
parabolic_character_rings = [WeylCharacterRing(CartanType(["A", index-1], ["C", rank-index])) for index in maximal_parabolic_indices]
parabolic_weight_rings = [WeightRing(parabolic_character_ring) for parabolic_character_ring in parabolic_character_rings]
parabolic_space_weyl_groups = [parabolic_weight_ring.space().weyl_group() for parabolic_weight_ring in parabolic_weight_rings]

pullback_bundle_weight = [2 for i in range(maximal_parabolic_indices[0])]
pullback_bundle_weight.append(1)

push_forward_bundles = [character_ring(*pullback_bundle_weight) for character_ring in parabolic_character_rings]

6. Get dimension of push forward bundles

In [7]:
for bundle in push_forward_bundles:
    print(f"The bundle {bundle} has weights {bundle.weight_multiplicities()} and rank {bundle.degree()}")

The bundle A2xC2(2,2,2,1,0) has weights {(2, 2, 2, 1, 0): 1, (2, 2, 2, 0, 1): 1, (2, 2, 2, 0, -1): 1, (2, 2, 2, -1, 0): 1} and rank 4
The bundle A3xC1(2,2,2,1,0) has weights {(2, 2, 2, 1, 0): 1, (2, 2, 1, 2, 0): 1, (2, 1, 2, 2, 0): 1, (1, 2, 2, 2, 0): 1} and rank 4


7. Calculate dual bundles

In [8]:
dual_bundles = [find_dual_rep(bundle) for bundle in push_forward_bundles]

for bundle in dual_bundles:
    print(f"One dual is {bundle}")

One dual is A2xC2(-2,-2,-2,1,0)
One dual is A3xC1(-1,-2,-2,-2,0)


8. Calculate exterior powers

In [9]:
exterior_power_bundles = [[dual_bundle.exterior_power(i) for i in range(1, bundle.degree()+1)] for dual_bundle in dual_bundles]

In [10]:
exterior_power_bundles

[[A2xC2(-2,-2,-2,1,0),
  A2xC2(-4,-4,-4,0,0) + A2xC2(-4,-4,-4,1,1),
  A2xC2(-6,-6,-6,1,0),
  A2xC2(-8,-8,-8,0,0)],
 [A3xC1(-1,-2,-2,-2,0),
  A3xC1(-3,-3,-4,-4,0),
  A3xC1(-5,-5,-5,-6,0),
  A3xC1(-7,-7,-7,-7,0)]]

9. Tensor exterior powers with line bundle and find cohomologies in entries of the Koszul complex

In [11]:
rho = sum(fundamental_weights)

twist = 1

print(f"Dimension of {(twist,)*(2*r-1)  + (0,)*(rank-2*r+1)} is {char_ring(twist*fundamental_weights[2*r-1]).degree()} \n")

irreps = [parabolic_character_rings[0](wt[0]) for i in range(len(exterior_power_bundles[0])) for wt in list(exterior_power_bundles[0][i])]

for irrep in irreps:
    wt = irrep.highest_weight()
    wt = [x[1] for x in wt]
    for i in range(rank-len(wt)):
        wt.append(0)

    for i in range(2*r-1):
        wt[i] += twist

    print(f"Finding cohomology of {wt}")
    print(borel_weil_bott("C", wt))
    print("\n")

Dimension of (1, 1, 1, 0, 0) is 110 

Finding cohomology of [-1, -1, -1, 1, 0]
No cohomology


Finding cohomology of [-3, -3, -3, 1, 1]
No cohomology


Finding cohomology of [-3, -3, -3, 0, 0]
No cohomology


Finding cohomology of [-5, -5, -5, 1, 0]
No cohomology


Finding cohomology of [-7, -7, -7, 0, 0]
No cohomology




In [12]:
rho = sum(fundamental_weights)

twist = 1

print(f"Dimension of {(twist,)*(2*r)  + (0,)*(rank-2*r)} is {char_ring(twist*fundamental_weights[2*r]).degree()} \n")

irreps = [parabolic_character_rings[0](wt[0]) for i in range(len(exterior_power_bundles[0])) for wt in list(exterior_power_bundles[0][i])]

for irrep in irreps:
    wt = irrep.highest_weight()
    wt = [x[1] for x in wt]
    for i in range(rank-len(wt)):
        wt.append(0)

    for i in range(2*r):
        wt[i] += twist

    print(f"Finding cohomology of {wt}")
    print(borel_weil_bott("C", wt))
    print("\n")

Dimension of (1, 1, 1, 1, 0) is 165 

Finding cohomology of [-1, -1, -1, 2, 0]
No cohomology


Finding cohomology of [-3, -3, -3, 2, 1]
No cohomology


Finding cohomology of [-3, -3, -3, 1, 0]
No cohomology


Finding cohomology of [-5, -5, -5, 2, 0]
No cohomology


Finding cohomology of [-7, -7, -7, 1, 0]
No cohomology


